### Step 1.  Data Preprocessing ###

In [ ]:
print("nice")

In [ ]:
# Header Files
import pandas as pd
import csv

In [ ]:
# Using Kaggle DataSet 
df = pd.read_csv('data/RAW_recipes.csv')
recipeIngredients = df[['name','ingredients']]
print("No. of recipes = ", len(recipeIngredients))
print((recipeIngredients[0:10]))

In [ ]:
#Using dataset from https://eightportions.com/datasets/Recipes/
#Contains recipe name and list of ingredients used
data1 = pd.read_json('data/recipes_raw_nosource_ar.json', orient = 'index')
recipeIg1 = data1[['title','ingredients']]
len1 = len(recipeIg1)

data2 = pd.read_json('data/recipes_raw_nosource_epi.json', orient = 'index')
recipeIg2 = data2[['title','ingredients']]
len2 = len(recipeIg2)

data3 = pd.read_json('data/recipes_raw_nosource_epi.json', orient = 'index')
recipeIg2 = data2[['title','ingredients']]
len2 = len(recipeIg2)


In [ ]:
#Data URL: https://www.kaggle.com/kaggle/recipe-ingredients-dataset(Data set with cuisines)

# Create URL to JSON file (alternatively this can be a filepath)
url = 'train.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient='columns')

ingredients_1 = df[['id','ingredients']]
# View the first ten rows
print(ingredients_1.head(10))
len(ingredients_1)

In [ ]:
#Data URL: https://www.kaggle.com/hugodarwood/epirecipes

# Create URL to JSON file (alternatively this can be a filepath)
url = 'full_format_recipes.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient='columns')

ingredients_2 = df[['title','ingredients']]
# View the first ten rows
print(ingredients_2.head(10))
print(len(ingredients_2))